In [2]:
import torch
from ultra import datasets
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= datasets.Epinions(root= "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data")
test= dataset[2]

# Assuming test.target_edge_index is defined and is a tensor of shape (2, num_edges)
user_ids = test.target_edge_index[0]  # row 0 contains user indices
unique_users, counts = torch.unique(user_ids, return_counts=True)
avg_interactions = counts.float().mean().item()
print("Average test interactions per user:", avg_interactions)



Average test interactions per user: 9.729940414428711


In [5]:
from ultra import datasets

dataset = datasets.Yelp18(root = "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data")
print (f"dataset[0].edge_index.shape{: dataset[0].edge_index.shape}")

dataset[0].edge_index.shapetorch.Size([2, 2194014])


In [1]:
import sqlite3
import pandas as pd

# File and datasets list
DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db" 
DATASETS = [
    "Epinions", "LastFM", "BookX", "Ml1m", "Gowalla",
    "Amazon_Beauty", "Amazon_Fashion", "Amazon_Men", "Amazon_Games", "Yelp18"
]

# Connect to the database and load the experiments table
conn = sqlite3.connect(DB_FILE)
df = pd.read_sql("SELECT * FROM experiments", conn)
conn.close()

# Filter for experiments that correspond to zero-shot (epochs=0) and fine tuning (epochs=1)
filtered_df = df[df['epochs'].isin([0, 1])]

# Exclude rows where the checkpoint is simply the dataset checkpoint (e.g. "LastFM.pth" for LastFM)
filtered_df = filtered_df[~filtered_df.apply(
    lambda row: row['ckpt'] == f"{row['dataset']}.pth", axis=1
)]

# List to collect top 3 results for each dataset and epoch type
results_list = []

for ds in DATASETS:
    # Filter the dataframe for the current dataset
    ds_df = filtered_df[filtered_df['dataset'] == ds]
    if ds_df.empty:
        continue  # Skip if no experiments exist for this dataset
    # Choose the metric: test_ndcg@20 for Gowalla and Yelp18, else test_hits@10
    metric = "test_ndcg@20" if ds in ["Gowalla", "Yelp18"] else "test_hits@10"
    
    # For both epoch types: 0 (zero-shot) and 1 (fine tuning)
    for epoch in [0, 1]:
        epoch_df = ds_df[ds_df['epochs'] == epoch]
        if not epoch_df.empty:
            # Sort by the chosen metric in descending order and take the top 3 entries
            top3 = epoch_df.sort_values(by=metric, ascending=False).head(3)
            results_list.append(top3)

# Concatenate all the top results
final_results = pd.concat(results_list)

# Save the results to a CSV file
final_results.to_csv("best_results_multi.csv", index=False)

# Optionally, print the results
print(final_results)


                        ckpt         dataset  epochs      bpe   FT   valid_mr  \
128        Amazon_Beauty.pth        Epinions     0.0   5130.0  0.0        NaN   
143                BookX.pth        Epinions     0.0   5130.0  0.0        NaN   
127   Beauty_Games_tuned.pth        Epinions     0.0   5130.0  0.0        NaN   
153               LastFM.pth        Epinions     1.0   5130.0  0.0        NaN   
278            Beau_Epin.pth        Epinions     1.0   5130.0  0.0        NaN   
271       Beau_Epin_Book.pth        Epinions     1.0   5130.0  0.0        NaN   
199         Amazon_Games.pth          LastFM     0.0    763.0  0.0        NaN   
216                BookX.pth          LastFM     0.0    763.0  0.0        NaN   
269            Beau_Epin.pth          LastFM     0.0    763.0  0.0        NaN   
206   Men_Epin_Gowa_Book.pth          LastFM     1.0    763.0  0.0        NaN   
224     inionsBeautyMl1m.pth          LastFM     1.0    763.0  0.0        NaN   
266       Beau_Epin_Book.pth

In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [13]:
import sqlite3
import pandas as pd

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"

# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Query to select the ckpts for dataset 'ml-1m' where epoch is 0 and epochs is 1
#DATASETS = [
   # "Epinions", "LastFM", "BookX", "Ml1m", "Gowalla",
   # "Amazon_Beauty", "Amazon_Fashion", "Amazon_Men", "Amazon_Games", "Yelp18"
#]
query = """
SELECT ckpt
FROM experiments
WHERE dataset = 'Yelp18'
  AND epochs = 1
"""
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Convert the ckpt column to a list and print it
ckpt_list = df['ckpt'].tolist()
print(ckpt_list)


SOTA metrics inserted successfully.


In [15]:
import sqlite3
import pandas as pd


DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db" 
# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Read all entries
df = pd.read_sql("SELECT * FROM experiments WHERE ckpt = 'SOTA'", conn)

# Close the connection
conn.close()

# Display the results

print(df.head())  # Use this if running in a terminal


   ckpt         dataset epochs   bpe    FT valid_mr valid_mrr valid_hits@1  \
0  SOTA            Ml1m   None  None  None     None      None         None   
1  SOTA           BookX   None  None  None     None      None         None   
2  SOTA        Epinions   None  None  None     None      None         None   
3  SOTA          LastFM   None  None  None     None      None         None   
4  SOTA  Amazon_Fashion   None  None  None     None      None         None   

  valid_hits@3 valid_hits@10 valid_ndcg@10 test_mr test_mrr test_hits@1  \
0         None          None          None    None     None        None   
1         None          None          None    None     None        None   
2         None          None          None    None     None        None   
3         None          None          None    None     None        None   
4         None          None          None    None     None        None   

  test_hits@3  test_hits@10  test_ndcg@10  valid_ndcg@20  test_ndcg@20  
0      

In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_all.yaml --gpus [0]

In [ ]:
%run script/run-debug.py -c config/recommender/notebook_cfg.yaml --dataset Gowalla --epochs 0 --bpe 0 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain/inionsBeautyMl1m.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
09:52:07   Random seed: 1024
09:52:07   Config file: config/recommender/notebook_cfg.yaml
09:52:07   {'checkpoint': '/itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain/inionsBeautyMl1m.pth',
 'dataset': {'class': 'Gowalla',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.03247182903513618,
                                                 'hidden_dims': [16,
                                                                 16,
                                                                 16,
                                                                 16,
                                       

We will evaluate vs all negatives


Processing...


size of train: train_edges: 810128
size of test: test_edges: 217242
Matched edges: 810128, Unmatched edges: 0
Matched edges: 217242, Unmatched edges: 0
size of train_edges_features_df: (810128, 5)
size of test_edges_features_df: (217242, 5)
Hi we are in the process_df method.
This is the shape of the original df (810128, 5)
This is the shape after we dropped cols (810128, 3)
Number of null rows: 0
Feature tensor shape(before constant_col_detection): torch.Size([810128, 4])
final Feature tensor shape: torch.Size([810128, 4])
process_df was sucessful
Hi we are in the process_df method.
This is the shape of the original df (217242, 5)
This is the shape after we dropped cols (217242, 3)
Number of null rows: 0
Feature tensor shape(before constant_col_detection): torch.Size([217242, 4])
final Feature tensor shape: torch.Size([217242, 4])
process_df was sucessful
size of train_edges_combined: 1425008
size of train_edges_combined_features: 1425008
size of train_target_edges: 712504
size of tra

Done!
09:53:28   Gowalla dataset
09:53:28   #train: 712504, #valid: 97624, #test: 217242


Graph validation passed!


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f98d9dca940>>
Traceback (most recent call last):
  File "/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Graph validation passed!
Graph validation passed!


In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Gowalla --epochs 0 --bpe 20000 --gpus "[0]" --ckpt null

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Epinions --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

In [7]:
import sqlite3

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"

# Connect to the database
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Manually insert Yelp18 data
yelp18_data = (
    "-", "Yelp18", 8, 17140, 0,  # ckpt, dataset, epochs, bpe, FT
    1006.4, 0.0366413, 0.0136183, 0.0315789, 0.073482, 0.0,  # valid_mr, valid_mrr, valid_hits@1, valid_hits@3, valid_hits@10, valid_ndcg@10
    1992.93, 0.017529, 0.00529698, 0.0132162, 0.034253,  # test_mr, test_mrr, test_hits@1, test_hits@3, test_hits@10
    0.0900301, 0.0676829  # valid_ndcg@20, test_ndcg@20
)

# Manually insert Gowalla data
gowalla_data = (
    "-", "Gowalla", 8, 22265, 0,  # ckpt, dataset, epochs, bpe, FT
    764.808, 0.080052, 0.0424691, 0.0786897, 0.149031, 0.0,  # valid_mr, valid_mrr, valid_hits@1, valid_hits@3, valid_hits@10, valid_ndcg@10
    70747.3, 1.41349e-05, 0, 0, 0,  # test_mr, test_mrr, test_hits@1, test_hits@3, test_hits@10
    0.1098, 0.0  # valid_ndcg@20, test_ndcg@20
)

# Corrected SQL query with double quotes around column names
insert_query = """
INSERT INTO experiments (
    ckpt, dataset, epochs, bpe, FT,
    valid_mr, valid_mrr, "valid_hits@1", "valid_hits@3", "valid_hits@10", "valid_ndcg@10",
    test_mr, test_mrr, "test_hits@1", "test_hits@3", "test_hits@10",
    "valid_ndcg@20", "test_ndcg@20"
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Execute the inserts
cursor.execute(insert_query, yelp18_data)
cursor.execute(insert_query, gowalla_data)

# Commit changes and close connection
conn.commit()
conn.close()

print("Entries for Yelp18 and Gowalla successfully added to the database.")


Entries for Yelp18 and Gowalla successfully added to the database.


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [2]:
import sqlite3
from pathlib import Path

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results_all_to_all.db"

# Ensure the directory exists
Path(DB_FILE).parent.mkdir(parents=True, exist_ok=True)

def create_database(db_path):
    """Create a new SQLite database and ensure the 'experiments' table exists."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Define the schema (adjust as needed)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS experiments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ckpt TEXT,
        dataset TEXT,
        epochs INTEGER,
        bpe INTEGER,
        FT INTEGER
    )
    """)

    conn.commit()
    conn.close()

# Call the function to create the database
create_database(DB_FILE)
print(f"Database created at {DB_FILE}")


Database created at //itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results_all_to_all.db


In [4]:
import pandas as pd
# Please dont delete counts the best results
# Load the filtered results CSV
df = pd.read_csv("best_results_multi.csv")

# Count occurrences for epochs = 0
epoch0_counts = df[df['epochs'] == 0]['ckpt'].value_counts()

# Count occurrences for epochs = 1
epoch1_counts = df[df['epochs'] == 1]['ckpt'].value_counts()

# Count occurrences overall (epochs 0 or 1)
overall_counts = df['ckpt'].value_counts()

# Display the counts
print("Checkpoint counts for epochs = 0:")
print(epoch0_counts)

print("\nCheckpoint counts for epochs = 1:")
print(epoch1_counts)

print("\nOverall checkpoint counts (epochs 0 or 1):")
print(overall_counts)



Checkpoint counts for epochs = 0:
ckpt
Amazon_Beauty.pth          3
BookX.pth                  3
Beauty_Games_tuned.pth     3
Beau_Epin.pth              3
Epinions.pth               3
Men_Epin_Gowa_Book.pth     3
Amazon_Games.pth           2
Amazon_Fashion.pth         2
Beau_Epin_Book_Game.pth    2
Beau_Epin_Book.pth         2
Yelp18.pth                 1
Gowalla.pth                1
inionsBeautyMl1m.pth       1
Amazon_Men.pth             1
Name: count, dtype: int64

Checkpoint counts for epochs = 1:
ckpt
Amazon_Games.pth           5
Beau_Epin.pth              4
Beau_Epin_Book.pth         4
Men_Epin_Gowa_Book.pth     3
Gowalla.pth                2
Ml1m.pth                   2
Amazon_Men.pth             2
Amazon_Beauty.pth          2
LastFM.pth                 1
inionsBeautyMl1m.pth       1
Epinions.pth               1
Beau_Epin_Book_Game.pth    1
Yelp18.pth                 1
Beauty_Games_tuned.pth     1
Name: count, dtype: int64

Overall checkpoint counts (epochs 0 or 1):
ckpt
Beau_Epi

In [3]:
import sqlite3
import pandas as pd

# Define database path
DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results_all_to_all.db"

def read_database_entries(db_path):
    """Connects to the SQLite database and retrieves all entries from the 'experiments' table."""
    try:
        # Connect to database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Check if table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='experiments'")
        table_exists = cursor.fetchone()
        
        if not table_exists:
            print("Table 'experiments' does not exist in the database.")
            return None
        
        # Read data into DataFrame
        df = pd.read_sql_query("SELECT * FROM experiments", conn)
        
        # Close connection
        conn.close()
        
        # Display the data
        print(df)

        return df
    
    except Exception as e:
        print(f"Error reading database: {e}")
        return None

# Execute function
read_database_entries(DB_FILE)


   id                ckpt dataset  epochs  bpe  FT valid_mr  valid_mrr  \
0   1   Amazon_Beauty.pth  LastFM       0  763   0     None   0.502169   
1   2        Epinions.pth  LastFM       0  763   0     None   0.456853   
2   3      Amazon_Men.pth  LastFM       0  763   0     None   0.429137   
3   4  Amazon_Fashion.pth  LastFM       0  763   0     None   0.266914   
4   5            Ml1m.pth  LastFM       0  763   0     None   0.312975   
5   6    Amazon_Games.pth  LastFM       0  763   0     None   0.486230   

   valid_hits@1  valid_hits@3  valid_hits@10  valid_ndcg@10 valid_ndcg@20  \
0      0.360044      0.582876       0.789791       0.289983          None   
1      0.311745      0.525796       0.764544       0.265768          None   
2      0.270033      0.507684       0.759605       0.250993          None   
3      0.150384      0.291438       0.515917       0.130463          None   
4      0.219539      0.362788       0.471460       0.132546          None   
5      0.333699    

,id,ckpt,dataset,epochs,bpe,FT,valid_mr,valid_mrr,valid_hits@1,valid_hits@3,valid_hits@10,valid_ndcg@10,valid_ndcg@20,test_mr,test_mrr,test_hits@1,test_hits@3,test_hits@10,test_ndcg@10,test_ndcg@20
0,1,Amazon_Beauty.pth,LastFM,0,763,0,None,0.502169,0.360044,0.582876,0.789791,0.289983,None,None,0.543926,0.403605,0.626434,0.820317,0.306830,None
1,2,Epinions.pth,LastFM,0,763,0,None,0.456853,0.311745,0.525796,0.764544,0.265768,None,None,0.500835,0.352266,0.580557,0.795740,0.283514,None
2,3,Amazon_Men.pth,LastFM,0,763,0,None,0.429137,0.270033,0.507684,0.759605,0.250993,None,None,0.473893,0.317313,0.553250,0.806663,0.268633,None
3,4,Amazon_Fashion.pth,LastFM,0,763,0,None,0.266914,0.150384,0.291438,0.515917,0.130463,None,None,0.306359,0.186783,0.336974,0.564173,0.141744,None
4,5,Ml1m.pth,LastFM,0,763,0,None,0.312975,0.219539,0.362788,0.471460,0.132546,None,None,0.360369,0.258875,0.407974,0.535227,0.146190,None
5,6,Amazon_Games.pth,LastFM,0,763,0,None,0.486230,0.333699,0.579034,0.787596,0.285585,None,None,0.531254,0.381759,0.617149,0.827963,0.303179,None


In [2]:
import sqlite3

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results_all_to_all.db"

def check_table_columns(db_path):
    """Checks the columns available in the 'experiments' table."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("PRAGMA table_info(experiments)")
    columns = [row[1] for row in cursor.fetchall()]

    conn.close()

    print("Columns in 'experiments' table:", columns)

check_table_columns(DB_FILE)


Columns in 'experiments' table: ['id', 'ckpt', 'dataset', 'epochs', 'bpe', 'FT', 'valid_mr', 'valid_mrr', 'valid_hits@1', 'valid_hits@3', 'valid_hits@10', 'valid_ndcg@10', 'valid_ndcg@20', 'test_mr', 'test_mrr', 'test_hits@1', 'test_hits@3', 'test_hits@10', 'test_ndcg@10', 'test_ndcg@20']


In [9]:
import sqlite3
import pandas as pd

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"
CSV_PATH = "/itet-stor/trachsele/net_scratch/tl4rec/report_data/Ml1m_all.csv"

# List of required checkpoints
required_ckpts = [
    "Beau_Epin.pth", "Beau_Epin_Book_Game.pth", "Men_Epin_Gowa_Book.pth",
    "Beau_Epin_Book.pth", "Beauty_Games_tuned.pth", "inionsBeautyMl1m.pth", "ETE", "SOTA"
]

# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Query to filter based on dataset and checkpoint
query = """
    SELECT * FROM experiments
    WHERE dataset = 'Ml1m'
    AND ckpt IN ({})
""".format(', '.join(['?'] * len(required_ckpts)))  # Use `.format()` to properly close the string

df = pd.read_sql(query, conn, params=required_ckpts)

# Close the connection
conn.close()

# Export the filtered DataFrame to a CSV file
df.to_csv(CSV_PATH, index=False)

print(f"Filtered data exported to {CSV_PATH}")


Filtered data exported to /itet-stor/trachsele/net_scratch/tl4rec/report_data/Ml1m_all.csv


In [ ]:
checkpoint_abbr = {
    "Beau_Epin.pth": "AB-Epi",
    "Beau_Epin_Book_Game.pth": "AB-Epi-Bx-AG",
    "Men_Epin_Gowa_Book.pth": "AM-Epi-Gw-Bx",
    "Beau_Epin_Book.pth": "AB-Epi-Bx",
    "Beauty_Games_tuned.pth": "AB-AG",
    "inionsBeautyMl1m.pth": "Epi-AB-M1"
}

In [ ]:
dataset_abbr = {
    "Amazon Beauty": "AB",
    "Amazon Games": "AG",
    "Amazon Fashion": "AF",
    "Amazon Men": "AM",
    "Epinions": "Epi",
    "ML-1M": "M1",
    "LastFM": "LF",
    "Gowalla": "Gw",
    "Yelp18": "Yp",
    "BookX": "Bx"
}
